# Prepare Data

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
# !git clone https://github.com/WongKinYiu/yolov7.git


In [4]:
!pip install roboflow

Defaulting to user installation because normal site-packages is not writeable


In [5]:
!pip install ultralytics

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for torch>=1.8.0 from https://files.pythonhosted.org/packages/51/03/1abad10990c76bee3703857b1617563b241f87d297ee466dbad922b0c308/torch-2.3.0-cp312-cp312-manylinux1_x86_64.whl.metadata
  Using cached torch-2.3.0-cp312-cp312-manylinux1_x86_64.whl.metadata (26 kB)
  Obtaining dependency information for nvidia-cuda-nvrtc-cu12==12.1.105 from https://files.pythonhosted.org/packages/b6/9f/c64c03f49d6fbc56196664d05dba14e3a561038a81a638eeb47f4d4cfd48/nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Obtaining dependency information for nvidia-cuda-runtime-cu12==12.1.105 from https://files.pythonhosted.org/packages/eb/d5/c68b1d2cdfcc59e72e8a5949a37ddb22ae6cade80cd4a57a84d4c8b55472/nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata
  Using cache

In [6]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [7]:
%cd /content/drive/MyDrive/NTI_GP

[Errno 2] No such file or directory: '/content/drive/MyDrive/NTI_GP'
/home/umakant/Documents/BTP


/home/umakant/.local/lib/python3.12/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [8]:

from roboflow import Roboflow
rf = Roboflow(api_key="dZtrnF63aDa6P0VhuGkj")
project = rf.workspace("fyp-pz5zs").project("violence-nonviolence-behavior")
version = project.version(34)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...


# Train Yolov8

In [9]:
from ultralytics import YOLO
model = YOLO('./content/violence_weights.pt')  # load a pretrained YOLOv8n detection model
# model.train(data='/content/drive/MyDrive/NTI_GP/Violence-NonViolence-Behavior-34/data.yaml', epochs=30)  # train the model

In [10]:
# from ultralytics import YOLO
# model = YOLO('yolov8n.pt')  # load a pretrained YOLOv8n detection model

# Video Analysis

In [11]:
!pip install transformers --quiet
!pip install ultralytics --quiet

In [12]:
import torch
import cv2
from ultralytics import YOLO

# Define path to video file
source = "./content/video1.webm"

# Load the YOLOv8 model and custom weights
model = YOLO('./content/violence_weights.pt')  # Start with a YOLOv8 base model (yolov8n.pt or similar)
# model.load('/content/violence_weights.pt')  # Load custom weights


# Run inference on the source
results = model(source, stream=True)  # generator of Results objects

### Extract Frames

In [13]:
import cv2
import os

# Initialize a list to hold frames (optional if you want to save them directly)
frames = []

# Specify the output folder where the frames will be saved
output_folder = 'extracted_frames'
os.makedirs(output_folder, exist_ok=True)  # Create the folder if it doesn't exist

# Specify how often to save frames, e.g., every 10th frame
frame_interval = 10  # Extract every 10th frame

# Process results list
for idx, result in enumerate(results):
    # Only extract and save every 'frame_interval' frame
    if idx % frame_interval == 0:
        # Extract result details (if needed)
        boxes = result.boxes
        masks = result.masks
        keypoints = result.keypoints
        probs = result.probs
        obb = result.obb

        # Use result.plot() to get the annotated image as a NumPy array
        annotated_frame = result.plot(boxes=False)
        frames.append(annotated_frame)

        # Construct the filename for the image
        image_filename = os.path.join(output_folder, f'frame_{idx:04d}.png')  # e.g., frame_0001.png

        # Save the frame as an image file
        cv2.imwrite(image_filename, annotated_frame)

print(f"Selected frames have been saved to the folder: {output_folder}")



video 1/1 (frame 1/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 4 nonviolences, 64.0ms
video 1/1 (frame 2/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 3 nonviolences, 3.1ms
video 1/1 (frame 3/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 5 nonviolences, 3.1ms
video 1/1 (frame 4/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 5 nonviolences, 3.1ms
video 1/1 (frame 5/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 4 nonviolences, 3.1ms
video 1/1 (frame 6/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 3 nonviolences, 3.1ms
video 1/1 (frame 7/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 3 nonviolences, 3.3ms
video 1/1 (frame 8/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 3 nonviolences, 3.1ms
video 1/1 (frame 9/19243) /home/umakant/Documents/BTP/content/video1.webm: 384x640 3 nonviolences, 3.1ms
video 1/1 (frame 10/19243) /home/umakant/Documents/BT

In [14]:
# !pip install transformers --quiet

### Generate Cpations

In [3]:
from transformers import AutoProcessor, Blip2ForConditionalGeneration,BlipForConditionalGeneration,BlipProcessor
import torch
# Load BLIP-2 processor and model for caption generation
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16).to("cuda")





preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [4]:
import os
import cv2
import torch
from ultralytics import YOLO
from transformers import AutoProcessor, Blip2ForConditionalGeneration
# Load YOLOv8 model for violence detection
yolo_model = YOLO('./content/violence_weights.pt')


# Function to generate caption using BLIP-2
def generate_caption(frame):
    inputs = processor(images=frame, return_tensors="pt").to("cuda", torch.float16)
    with torch.no_grad():
        generated_ids = caption_model.generate(**inputs, max_length=50)
        caption = processor.decode(generated_ids[0], skip_special_tokens=True)
    return caption

# Function to overlay caption on the frame
def overlay_caption_on_frame(frame, caption):
    # Set font and positioning for the caption
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    caption_color = (255, 255, 255)  # White text
    text_position = (50, 50)  # Position of the text on the frame

    # Draw the caption on the frame
    cv2.putText(frame, caption, text_position, font, font_scale, caption_color, font_thickness, cv2.LINE_AA)
    return frame

# Define folders and process results
output_folder = 'processed_frames'
captions_file = 'captions.txt3'  # Ensure correct path for captions
os.makedirs(output_folder, exist_ok=True)

# Open the file to save captions
with open(captions_file, 'w') as f:
    # Load frames from the directory
    frames_folder = './content/extracted_frames'
    frame_files = [os.path.join(frames_folder, file) for file in os.listdir(frames_folder) if file.endswith(('.png', '.jpg'))]

    for idx, frame_path in enumerate(frame_files):
        # Load the frame
        frame = cv2.imread(frame_path)

        if frame is None:
            print(f"Error loading frame {frame_path}. Skipping.")
            continue

        # Generate a caption using BLIP-2
        caption = generate_caption(frame)

        if caption:  # Check if the caption is not empty
            print(f"Generated caption for frame {idx}: {caption}")

            # Overlay caption on the frame
            annotated_frame_with_caption = overlay_caption_on_frame(frame, caption)

            # Save the frame with caption overlay
            frame_filename = os.path.join(output_folder, f'frame_{idx:04d}.png')
            cv2.imwrite(frame_filename, annotated_frame_with_caption)

            # Save the caption in the captions file
            f.write(f"Frame {idx:04d}: {caption}\n")
        else:
            print(f"Warning: No caption generated for frame {idx}")

print(f"Processed frames saved to {output_folder} and captions saved to {captions_file}")

Generated caption for frame 0: several men in suits and ties walking in a room
Generated caption for frame 1: several men in suits and ties walking down a hallway
Generated caption for frame 2: arafed man in a suit and tie walking down a hallway
Generated caption for frame 3: arafed image of two men in suits walking in a room
Generated caption for frame 4: two men in suits and ties walking down a hallway
Generated caption for frame 5: there are two men in suits walking together in a room
Generated caption for frame 6: two men in suits and ties are walking in a dark room
Generated caption for frame 7: arafed image of two men in suits walking in a dark room
Generated caption for frame 8: arafed image of a man in a suit and tie walking down a hallway
Generated caption for frame 9: arafed image of a man in a suit and tie walking with other men
Generated caption for frame 10: arafed image of a man in a suit and tie standing next to another man
Generated caption for frame 11: arafed image of